# Template

YouTube playlists:

jazz - https://www.youtube.com/watch?v=RPfFhfSuUZ4&list=PL8F6B0753B2CCA128  

country - https://www.youtube.com/watch?v=zXDAYlhdkyg&list=PL3oW2tjiIxvQW6c-4Iry8Bpp3QId40S5S  

hip_hop - https://www.youtube.com/playlist?list=PLAPo1R_GVX4IZGbDvUH60bOwIOnZplZzM  

metal - https://www.youtube.com/playlist?list=PLfY-m4YMsF-OM1zG80pMguej_Ufm8t0VC  

classical - https://www.youtube.com/watch?v=kSE15tLBdso&list=PLRb-5mC4V_Lop8KLXqSqMv4_mqw5M9jjW  

electronic - https://www.youtube.com/watch?v=pvuN_WvF1to&list=PLDDAxmBan0BKeIxuYWjMPBWGXDqNRaW5S

In [ ]:
from __future__ import unicode_literals
import youtube_dl

genres = {'country': 'https://www.youtube.com/watch?v=zXDAYlhdkyg&list=PL3oW2tjiIxvQW6c-4Iry8Bpp3QId40S5S', 
          'jazz': 'https://www.youtube.com/watch?v=RPfFhfSuUZ4&list=PL8F6B0753B2CCA128', 
          'hip_hop': 'https://www.youtube.com/playlist?list=PLAPo1R_GVX4IZGbDvUH60bOwIOnZplZzM', 
          'classical': 'https://www.youtube.com/watch?v=kSE15tLBdso&list=PLRb-5mC4V_Lop8KLXqSqMv4_mqw5M9jjW', 
          'metal': 'https://www.youtube.com/playlist?list=PLfY-m4YMsF-OM1zG80pMguej_Ufm8t0VC', 
          'electronic': 'https://www.youtube.com/watch?v=pvuN_WvF1to&list=PLDDAxmBan0BKeIxuYWjMPBWGXDqNRaW5S'
 }

for genre, playlist in genres.items():
    
    path = 'songs/' + genre + '/%(title)s.%(ext)s'
    
    ydl_opts = {
        'format': 'bestaudio/best',
        'ignoreerrors': True,
        'playlistend': 110,
        'nooverwrites': True,
        'outtmpl': path,
        'postprocessors': [{
            'key': 'FFmpegExtractAudio',
            'preferredcodec': 'mp3',
            'preferredquality': '192',
        }],
    }

    with youtube_dl.YoutubeDL(ydl_opts) as ydl:
        ydl.download([playlist])

In [3]:
import csv
import os
import librosa
import numpy as np

header = 'name tempo beats chroma_stft rmse spec_cent spec_bw rolloff zcr'.split()
header += ['mfcc_' + str(i) for i in range(1,12)] + ['label']
file = open('data.csv', 'w', newline='')

with file:
    writer = csv.writer(file)
    writer.writerow(header)
    genres = ['country', 'jazz'] #, 'metal', 'hip_hop', 'electronic', 'classical']

for genre in genres:
    for filename in os.listdir('./songs/' + genre):
        if filename.endswith('.mp3'):
            songname = f'./songs/{genre}/{filename}'
            y, sr = librosa.load(songname, mono=True, duration=180, sr=None)
            tempo, beats = librosa.beat.beat_track(y=y, sr=sr)
            chroma_stft = librosa.feature.chroma_stft(y=y, sr=sr)
            rmse = librosa.feature.rmse(y=y)
            spec_cent = librosa.feature.spectral_centroid(y=y, sr=sr)
            spec_bw = librosa.feature.spectral_bandwidth(y=y, sr=sr)
            rolloff = librosa.feature.spectral_rolloff(y=y, sr=sr)
            zcr = librosa.feature.zero_crossing_rate(y)
            mfcc = librosa.feature.mfcc(y=y, sr=sr, n_mfcc=12)
            label = genre.title()

            to_append = f'{tempo}, {beats.shape[0]}, {np.mean(chroma_stft)}, {np.mean(rmse)},'
            to_append += f'{np.mean(spec_cent)}, {np.mean(spec_bw)}, {np.mean(rolloff)}, {np.mean(zcr)}'    

            for e in mfcc[1:]:
                to_append += f',{np.mean(e)}'

            to_append += f',{label}'
            file = open('data.csv', 'a', newline='')

            with file:
                writer = csv.writer(file)
                song_row = [filename] + to_append.split(',')
                writer.writerow(song_row)

In [8]:
import pandas as pd

pat = pd.read_csv('classical_electronic.csv')
zander = pd.read_csv('metal_hiphop.csv')
zander['name'] = zander['Name']
zander.drop('Name', axis=1, inplace=True)
kaspar = pd.read_csv('country_jazz.csv')

df = pd.concat([pat, zander, kaspar], ignore_index=True, sort=False)
df.shape

(648, 21)

In [12]:
df.to_csv('data.csv')